In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings as wn
import sklearn.preprocessing as skpre
import category_encoders as ce

wn.simplefilter( "ignore" )

# Preparamos los Sets de Training

In [2]:
loc_ts = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Training Sets"

In [3]:
ins_18_20 = pd.read_csv( loc_ts + "\\ins_18_20.csv" ); 
ins_21_23 = pd.read_csv( loc_ts + "\\ins_21_23.csv" ); 
ins_24_26 = pd.read_csv( loc_ts + "\\ins_24_26.csv" ); 

In [4]:
evt_18_20 = pd.read_csv( loc_ts + "\\evt_18_20.csv" );
evt_21_23 = pd.read_csv( loc_ts + "\\evt_21_23.csv" );
evt_24_26 = pd.read_csv( loc_ts + "\\evt_24_26.csv" );

In [5]:
auc_18_20 = pd.read_csv( loc_ts + "\\auc_18_20.csv" ); 
auc_21_23 = pd.read_csv( loc_ts + "\\auc_21_23.csv" );
auc_24_26 = pd.read_csv( loc_ts + "\\auc_24_26.csv" );

In [6]:
clk_18_20 = pd.read_csv( loc_ts + "\\clk_18_20.csv" );
clk_21_23 = pd.read_csv( loc_ts + "\\clk_21_23.csv" );
clk_24_26 = pd.read_csv( loc_ts + "\\clk_24_26.csv" );

# Genero los Sets con el primer install

In [7]:
auc_18_20["date"] =  pd.to_datetime( auc_18_20["date"] ); auc_18_20["_st"] = auc_18_20["date"] - dt.datetime( year = 2019, month = 4, day = 18 ); auc_18_20["_st"] = auc_18_20["_st"].dt.total_seconds();
auc_21_23["date"] =  pd.to_datetime( auc_21_23["date"] ); auc_21_23["_st"] = auc_21_23["date"] - dt.datetime( year = 2019, month = 4, day = 21 ); auc_21_23["_st"] = auc_21_23["_st"].dt.total_seconds(); 
auc_24_26["date"] =  pd.to_datetime( auc_24_26["date"] ); auc_24_26["_st"] = auc_24_26["date"] - dt.datetime( year = 2019, month = 4, day = 24 ); auc_24_26["_st"] = auc_24_26["_st"].dt.total_seconds(); 

In [8]:
auc_18_20_first_auc = auc_18_20.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_21_23_first_auc = auc_21_23.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )
auc_24_26_first_auc = auc_24_26.sort_values( by = ["device_id","_st"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id": "ref_hash"} )

# Sacamos los Features

In [9]:
loc_ftr = r"D:\FacundoTorraca\Documents\TP2_Machine_Learning\Features\ftr_auc"

In [10]:
rh_18_20 = auc_18_20_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que usamos para entrenar
rh_21_23 = auc_21_23_first_auc[ ["ref_hash"] ] #RH unicos que coinciden con los datos que queremos predecir

tg_18_20 = auc_18_20_first_auc[ ["_st"] ]
tg_21_23 = auc_21_23_first_auc[ ["_st"] ]

## =========================================================================

## <span style="color:green"> **Hora de la realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. Por ejemplo que las 00 hs tengan un valor similar a 2

In [53]:
hora_auc_18_20 = rh_18_20.copy()
hora_auc_21_23 = rh_21_23.copy()

hora_auc_18_20["hour"] = auc_18_20_first_auc["date"].dt.hour; hora_auc_18_20["hour"] = hora_auc_18_20["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
hora_auc_21_23["hour"] = auc_21_23_first_auc["date"].dt.hour; hora_auc_21_23["hour"] = hora_auc_21_23["hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

hora_auc_18_20.to_csv( loc_ftr + "\\hora_auc_18_20.csv", index = False )
hora_auc_21_23.to_csv( loc_ftr + "\\hora_auc_21_23.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en las subastas** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [99]:
main_ahr_18_20 = rh_18_20.copy()
main_ahr_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20[["device_id"]]; hour_mode_18_20["hour"] = pd.to_datetime( auc_18_20["date"] ).dt.hour
hour_mode_21_23 = auc_21_23[["device_id"]]; hour_mode_21_23["hour"] = pd.to_datetime( auc_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["device_id", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["device_id","count"], ascending = False ).drop_duplicates( subset = ["device_id"], keep = "first" ).rename( columns = {"device_id":"ref_hash"} ).drop("count",axis = 1)

main_ahr_18_20 = main_ahr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ahr_21_23 = main_ahr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ahr_18_20["main_hour"] =  main_ahr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ahr_21_23["main_hour"] =  main_ahr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ahr_18_20.to_csv( loc_ftr + "\\main_ahr_18_20.csv", index = False )
main_ahr_21_23.to_csv( loc_ftr + "\\main_ahr_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Cantidad de Clicks en la ventana anterior** </span> 

In [69]:
cant_clk_18_20 = rh_18_20.copy()
cant_clk_21_23 = rh_21_23.copy()

cant_clk_18_20 = cant_clk_18_20.merge( clk_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_clk_21_23 = cant_clk_21_23.merge( clk_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_clk", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_clk_18_20.fillna( 0, inplace = True )
cant_clk_21_23.fillna( 0, inplace = True )

cant_clk_18_20.to_csv( loc_ftr + "\\cant_clk_18_20.csv", index = False )
cant_clk_21_23.to_csv( loc_ftr + "\\cant_clk_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones por dispositivo en la ventana previa a la conversion** </span> 

In [73]:
cant_ins_18_20 = rh_18_20.copy()
cant_ins_21_23 = rh_21_23.copy()

cant_ins_18_20 = cant_ins_18_20.merge( ins_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_ins_21_23 = cant_ins_21_23.merge( ins_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_ins", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_ins_18_20.fillna( 0, inplace = True )
cant_ins_21_23.fillna( 0, inplace = True )

cant_ins_18_20.to_csv( loc_ftr + "\\cant_ins_18_20.csv", index = False )
cant_ins_21_23.to_csv( loc_ftr + "\\cant_ins_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de subastas por dispositivo en la ventana previa a la conversion** </span> 

In [86]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "cant_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20.to_csv( loc_ftr + "\\cant_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\cant_auc_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de eventos por dispositivo en la ventana previa a la conversion** </span> 

In [92]:
cant_evt_18_20 = rh_18_20.copy()
cant_evt_21_23 = rh_21_23.copy()

cant_evt_18_20 = cant_evt_18_20.merge( evt_18_20["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_evt_21_23 = cant_evt_21_23.merge( evt_21_23["ref_hash"].value_counts().to_frame().reset_index().rename( columns = {"ref_hash": "cant_evt", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_evt_18_20.fillna( 0, inplace = True )
cant_evt_21_23.fillna( 0, inplace = True )

cant_evt_18_20.to_csv( loc_ftr + "\\cant_evt_18_20.csv", index = False )
cant_evt_21_23.to_csv( loc_ftr + "\\cant_evt_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Ref Type del dispositivo que realizo la instalacion** </span> 

#### <span style="color:orange"> **OneHotEncoding** </span> 

In [ ]:
ref_type_18_20 = rh_18_20.copy()
ref_type_21_23 = rh_21_23.copy()

rt_ohe_18_20 = pd.get_dummies( auc_18_20_first_auc["ref_type_id"] )
rt_ohe_21_23 = pd.get_dummies( auc_21_23_first_auc["ref_type_id"] )

ref_type_18_20 = pd.concat( [ref_type_18_20, rt_ohe_18_20], axis = 1 )
ref_type_21_23 = pd.concat( [ref_type_21_23, rt_ohe_21_23], axis = 1 )

ref_type_18_20.to_csv( loc_ftr + "\\ref_type_18_20.csv", index = False )
ref_type_21_23.to_csv( loc_ftr + "\\ref_type_21_23.csv", index = False )

## <span style="color:green"> **Source que recibio mas instalaciones por dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la fuente principal de algun dispositivo)

In [114]:
srce_auc_18_20 = rh_18_20.copy()
srce_auc_21_23 = rh_21_23.copy()

main_source_18_20 = auc_18_20.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()
main_source_21_23 = auc_21_23.groupby( ["device_id", "source_id"] ).agg( {"source_id":"count"} ).rename( columns = {"source_id":"cant_auc"} ).reset_index()

main_source_18_20 = main_source_18_20.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_18_20["cant_auc"] )
main_source_21_23 = main_source_21_23.sort_values( by = ["device_id","cant_auc"], ascending = True ).drop_duplicates( subset = ["device_id"], keep = "last" ); del( main_source_21_23["cant_auc"] )

srce_auc_18_20 = srce_auc_18_20.merge( main_source_18_20.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_18_20.fillna( "no_font", inplace = True )
srce_auc_21_23 = srce_auc_21_23.merge( main_source_21_23.rename(columns = {"device_id":"ref_hash"}), how = "left", on = "ref_hash" ); srce_auc_21_23.fillna( "no_font", inplace = True )

srce_auc_18_20["to_count"] = 1; srce_auc_18_20["source_id"] = srce_auc_18_20[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_18_20); del(srce_auc_18_20["to_count"])
srce_auc_21_23["to_count"] = 1; srce_auc_21_23["source_id"] = srce_auc_21_23[["source_id", "to_count"]].groupby("source_id").transform( "sum" ) / len(srce_auc_21_23); del(srce_auc_21_23["to_count"])

srce_auc_18_20.to_csv( loc_ftr + "\\srce_auc_18_20.csv", index = False )
srce_auc_21_23.to_csv( loc_ftr + "\\srce_auc_21_23.csv", index = False )

## <span style="color:green"> **Ref Hash** </span> 


#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de apariciones en las subastas en la ventana anterior)

In [47]:
rh_encod_18_20 = rh_18_20.copy()
rh_encod_21_23 = rh_21_23.copy()

appces_18_20_auc = auc_18_20["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )
appces_21_23_auc = auc_21_23["device_id"].value_counts().reset_index().rename( columns = {"index":"ref_hash", "device_id":"appearances"} )

appces_18_20_auc["ref_hash_mean"] = appces_18_20_auc["appearances"] / len( auc_18_20 ); appces_18_20_auc.drop( "appearances", axis = 1, inplace = True )
appces_21_23_auc["ref_hash_mean"] = appces_21_23_auc["appearances"] / len( auc_21_23 ); appces_21_23_auc.drop( "appearances", axis = 1, inplace = True )

rh_encod_18_20 = rh_encod_18_20.merge( appces_18_20_auc, how = "left", on = "ref_hash" )
rh_encod_21_23 = rh_encod_21_23.merge( appces_21_23_auc, how = "left", on = "ref_hash" )

rh_encod_18_20.to_csv( loc_ftr + "\\rh_encod_18_20.csv", index = False )
rh_encod_21_23.to_csv( loc_ftr + "\\rh_encod_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Dia de la semana realizacion de la primera subasta** </span> 

Le aplicamos la transformacion sin( (hora * pi)/6 ) para agregarle periodicidad. Por ejemplo 0 (lunes) tengan un valor similar a 6 (domingo)

In [108]:
sdia_auc_18_20 = rh_18_20.copy()
sdia_auc_21_23 = rh_21_23.copy()

sdia_auc_18_20["hour"] = auc_18_20_first_auc["date"].dt.hour; sdia_auc_18_20["hour"] = sdia_auc_18_20["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )
sdia_auc_21_23["hour"] = auc_21_23_first_auc["date"].dt.hour; sdia_auc_21_23["hour"] = sdia_auc_21_23["hour"].apply( lambda x: np.sin( (x*np.pi)/6 ) )

sdia_auc_18_20.to_csv( loc_ftr + "\\sdia_auc_18_20.csv", index = False )
sdia_auc_21_23.to_csv( loc_ftr + "\\sdia_auc_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Tipo de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese tipo sobre el total de eventos realizados)

In [26]:
kind_evt_18_20 = rh_18_20.copy()
kind_evt_21_23 = rh_21_23.copy()

kind_evt_18_20 = rh_18_20.copy()
kind_evt_21_23 = rh_21_23.copy()

main_kind_evt_18_20 = evt_18_20.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()
main_kind_evt_21_23 = evt_21_23.groupby( by = ["ref_hash","kind"] ).agg( {"kind":"count"} ).rename( columns = {"kind":"cant_evt"} ).reset_index()

main_kind_evt_18_20 = main_kind_evt_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_18_20["cant_evt"] )
main_kind_evt_21_23 = main_kind_evt_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_kind_evt_21_23["cant_evt"] ) 

kind_evt_18_20 = kind_evt_18_20.merge( main_kind_evt_18_20, how = "left", on = "ref_hash" )
kind_evt_21_23 = kind_evt_21_23.merge( main_kind_evt_21_23, how = "left", on = "ref_hash" )

cant_kind_evt_18_20 = evt_18_20[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )
cant_kind_evt_21_23 = evt_21_23[["ref_hash","kind"]].groupby( "kind" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_kind"} )

kind_evt_18_20 = kind_evt_18_20.merge( cant_kind_evt_18_20, how = "left", on = "kind" ).drop( "kind", axis = 1 )
kind_evt_21_23 = kind_evt_21_23.merge( cant_kind_evt_21_23, how = "left", on = "kind" ).drop( "kind", axis = 1 )

kind_evt_18_20["kind_mean"] = kind_evt_18_20["cant_kind"] / len(evt_18_20); kind_evt_18_20.fillna( kind_evt_18_20["kind_mean"].isnull().sum() / len(evt_18_20), inplace = True ); del(kind_evt_18_20["cant_kind"])
kind_evt_21_23["kind_mean"] = kind_evt_21_23["cant_kind"] / len(evt_21_23); kind_evt_21_23.fillna( kind_evt_21_23["kind_mean"].isnull().sum() / len(evt_21_23), inplace = True ); del(kind_evt_21_23["cant_kind"])

kind_evt_18_20.to_csv( loc_ftr + "\\kind_evt_18_20.csv", index = False )
kind_evt_21_23.to_csv( loc_ftr + "\\kind_evt_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Promedio del TimeToClick es un anuncio** </span> 

In [ ]:
ttc_mean_18_20 = rh_18_20.copy()
ttc_mean_21_23 = rh_21_23.copy()

ttc_by_rh_18_20 = clk_18_20[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_18_20 = ttc_by_rh_18_20.reset_index()
ttc_by_rh_21_23 = clk_21_23[ ["ref_hash", "timeToClick"] ].groupby( "ref_hash" ).agg( "mean" ); ttc_by_rh_21_23 = ttc_by_rh_21_23.reset_index()

ttc_mean_18_20 = ttc_mean_18_20.merge( ttc_by_rh_18_20, how = "left", on = "ref_hash" )
ttc_mean_21_23 = ttc_mean_21_23.merge( ttc_by_rh_21_23, how = "left", on = "ref_hash" )

Casi todos los valores nulos!



## =========================================================================

## <span style="color:green"> **Modelo del dispositivo** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el ST para el promedio)

In [ ]:
#Me quedo con los disppsitivos que coinciden en ambas ventanas (de manera que pueda obtener su SC)
rh_18_20_21_23_dm = rh_18_20.loc[ rh_18_20["ref_hash"].isin( rh_21_23["ref_hash"] ) ]

#Le agregamos el valor de "ST" a los dispositivos
rh_18_20_21_23_dm = rh_18_20_21_23_dm.merge( auc_21_23_first_auc[ ["ref_hash","_st"] ], how = "left", on = "ref_hash" )

#Obtenermos el modelo de dispositivo utilizado por cada RefHash 
dm_18_20_21_23_evt = evt_18_20.loc[ evt_18_20["ref_hash"].isin( rh_18_20_21_23_dm["ref_hash"] ), ["ref_hash","device_model"] ]; 
dm_18_20_21_23_evt.rename( columns = {"device_model":"dev_mod_evt"}, inplace = True )

dm_not_in_evt = dm_18_20_21_23_evt.loc[ dm_18_20_21_23_evt["dev_mod_evt"].isnull() ]["ref_hash"]

#Obtenemos los que no se pudieron del dataset de evetnos del data set de installs
dm_18_20_21_23_ins = ins_18_20.loc[ ins_18_20["ref_hash"].isin( dm_not_in_evt ), ["ref_hash","device_model"] ]; 
dm_18_20_21_23_ins.rename( columns = {"device_model":"dev_mod_ins"}, inplace = True )

#Mergemos los dispositivos con los ref_hash
rh_18_20_21_23_dm = rh_18_20_21_23_dm.merge( dm_18_20_21_23_evt, how = "left", on = "ref_hash" )
rh_18_20_21_23_dm = rh_18_20_21_23_dm.merge( dm_18_20_21_23_ins, how = "left", on = "ref_hash" )

#Ponemos todos los dispositivos en una sola columna
rh_18_20_21_23_dm["dev_mod"] = np.where( rh_18_20_21_23_dm['dev_mod_evt'].isnull(), rh_18_20_21_23_dm['dev_mod_ins'], rh_18_20_21_23_dm['dev_mod_evt']); 
del( rh_18_20_21_23_dm["dev_mod_evt"] )
del( rh_18_20_21_23_dm["dev_mod_ins"] )

#Dropeamos los duplicados
rh_18_20_21_23_dm = rh_18_20_21_23_dm.sort_values( ["ref_hash","dev_mod"] ).drop_duplicates( subset = ["ref_hash"], keep = "first" )

Sobran menos de 60000 valores no nulos de un total de 230673

## <span style="color:green"> **Lenguage del dispositivo** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usando el ST para el promedio)

In [ ]:
#Me quedo con los disppsitivos que coinciden en ambas ventanas (de manera que pueda obtener su SC)
rh_18_20_21_23_dl= rh_18_20.loc[ rh_18_20["ref_hash"].isin( rh_21_23["ref_hash"] ) ]

#Le agregamos el valor de "ST" a los dispositivos
rh_18_20_21_23_dl = rh_18_20_21_23_dl.merge( auc_21_23_first_auc[ ["ref_hash","_st"] ], how = "left", on = "ref_hash" )

#Obtenermos el modelo de dispositivo utilizado por cada RefHash 
dl_18_20_21_23_evt = evt_18_20.loc[ evt_18_20["ref_hash"].isin( rh_18_20_21_23_dm["ref_hash"] ), ["ref_hash","device_language"] ]; 
dl_18_20_21_23_evt.rename( columns = {"device_language":"dev_lan_evt"}, inplace = True )

dl_not_in_evt = dl_18_20_21_23_evt.loc[ dl_18_20_21_23_evt["dev_lan_evt"].isnull() ]["ref_hash"]

#Obtenemos los que no se pudieron del dataset de evetnos del data set de installs
dl_18_20_21_23_ins = ins_18_20.loc[ ins_18_20["ref_hash"].isin( dm_not_in_evt ), ["ref_hash","device_language"] ]; 
dl_18_20_21_23_ins.rename( columns = {"device_language":"dev_lan_ins"}, inplace = True )

#Mergemos los dispositivos con los ref_hash
rh_18_20_21_23_dl = rh_18_20_21_23_dl.merge( dl_18_20_21_23_evt, how = "left", on = "ref_hash" )
rh_18_20_21_23_dl = rh_18_20_21_23_dl.merge( dl_18_20_21_23_ins, how = "left", on = "ref_hash" )

#Ponemos todos los dispositivos en una sola columna
rh_18_20_21_23_dl["dev_lan"] = np.where( rh_18_20_21_23_dl['dev_lan_evt'].isnull(), rh_18_20_21_23_dl['dev_lan_ins'], rh_18_20_21_23_dl['dev_lan_evt']); 
del( rh_18_20_21_23_dl["dev_lan_evt"] )
del( rh_18_20_21_23_dl["dev_lan_ins"] )

#Dropeamos los duplicados
rh_18_20_21_23_dm = rh_18_20_21_23_dm.sort_values( ["ref_hash","dev_mod"] ).drop_duplicates( subset = ["ref_hash"], keep = "first" )


Sobran menos de 60000 valores no nulos de un total de 230673

## =========================================================================

## <span style="color:green"> **Cantidad de eventos atribuidos por dispositivo** </span> 

In [ ]:
cevt_atr_18_20 = rh_18_20.copy()
cevt_atr_21_23 = rh_21_23.copy()

cant_atr_evt_18_20 = evt_18_20[["ref_hash", "attributed"]]; cant_atr_evt_18_20["attributed"] = cant_atr_evt_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_evt_21_23 = evt_21_23[["ref_hash", "attributed"]]; cant_atr_evt_21_23["attributed"] = cant_atr_evt_21_23["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_evt_18_20 = cant_atr_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_evt_21_23 = cant_atr_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

cevt_atr_18_20 = cevt_atr_18_20.merge( cant_atr_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cevt_atr_21_23 = cevt_atr_21_23.merge( cant_atr_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cevt_atr_18_20.to_csv( loc_ftr + "\\cevt_atr_18_20.csv", index = False )
cevt_atr_21_23.to_csv( loc_ftr + "\\cevt_atr_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de eventos realizados con WiFi** </span> 

In [ ]:
wifi_evt_18_20 = rh_18_20.copy()
wifi_evt_21_23 = rh_21_23.copy()

cant_wifi_evt_18_20 = evt_18_20[["ref_hash", "wifi"]]; cant_wifi_evt_18_20["wifi"] = cant_wifi_evt_18_20["wifi"].apply( lambda x: 1 if x else 0 );
cant_wifi_evt_21_23 = evt_21_23[["ref_hash", "wifi"]]; cant_wifi_evt_21_23["wifi"] = cant_wifi_evt_21_23["wifi"].apply( lambda x: 1 if x else 0 );

cant_wifi_evt_18_20 = cant_wifi_evt_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_wifi_evt_21_23 = cant_wifi_evt_21_23.groupby( "ref_hash" ).agg( "sum" )

wifi_evt_18_20 = wifi_evt_18_20.merge( cant_wifi_evt_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
wifi_evt_21_23 = wifi_evt_21_23.merge( cant_wifi_evt_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

wifi_evt_18_20.to_csv( loc_ftr + "\\wifi_evt_18_20.csv", index = False )
wifi_evt_21_23.to_csv( loc_ftr + "\\wifi_evt_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Horario del primer evento en ese ventana** </span> 

Al agregarle la transformacion senoidal para agregarle perdiodicidad a la hora, la prediccion del algoritmo fue peor por lo que decidimos no agregarla

In [ ]:
hr_f_evt_18_20 = rh_18_20.copy()
hr_f_evt_21_23 = rh_21_23.copy()

first_evt_hour_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_evt_hour_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_evt_hour_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_evt_hour_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_evt_hour_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_evt_hour_18_20["hour_frt_evt"] = pd.to_datetime( first_evt_hour_18_20["date"] ).dt.hour
first_evt_hour_21_23["hour_frt_evt"] = pd.to_datetime( first_evt_hour_21_23["date"] ).dt.hour

first_evt_hour_18_20.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )
first_evt_hour_21_23.drop( ["date", "time_to_frt_evt"], axis = 1, inplace = True )

hr_f_evt_18_20 = hr_f_evt_18_20.merge( first_evt_hour_18_20, how = "left", on = "ref_hash" )
hr_f_evt_21_23 = hr_f_evt_21_23.merge( first_evt_hour_21_23, how = "left", on = "ref_hash" )

#hr_f_evt_18_20["hour_frt_evt"] = hr_f_evt_18_20["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )
#hr_f_evt_21_23["hour_frt_evt"] = hr_f_evt_21_23["hour_frt_evt"].apply( lambda x: np.sin( (x *np.pi)/24 ) )

hr_f_evt_18_20.to_csv( loc_ftr + "\\hr_f_evt_18_20.csv", index = False )
hr_f_evt_21_23.to_csv( loc_ftr + "\\hr_f_evt_21_23.csv", index = False )

## <span style="color:green"> **Hora mas popular del usuario en los eventos** </span> 

Le aplicamos la transformacion sin( (hora * pi)/24 ) para agregarle periodicidad. 

In [ ]:
main_ehr_18_20 = rh_18_20.copy()
main_ehr_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20[["ref_hash"]]; hour_mode_18_20["hour"] = pd.to_datetime( evt_18_20["date"] ).dt.hour
hour_mode_21_23 = evt_21_23[["ref_hash"]]; hour_mode_21_23["hour"] = pd.to_datetime( evt_21_23["date"] ).dt.hour

hour_mode_18_20 = hour_mode_18_20.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()
hour_mode_21_23 = hour_mode_21_23.groupby( ["ref_hash", "hour"] ).agg( { "hour":"count" } ).rename( columns = {"hour":"count"} ).reset_index()

hour_mode_18_20 = hour_mode_18_20.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)
hour_mode_21_23 = hour_mode_21_23.sort_values( ["ref_hash","count"], ascending = False ).drop_duplicates( subset = ["ref_hash"], keep = "first" ).drop("count",axis = 1)

main_ehr_18_20 = main_ehr_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )
main_ehr_21_23 = main_ehr_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" ).rename( columns = {"hour":"main_hour"} )

main_ehr_18_20["main_hour"] =  main_ehr_18_20["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )
main_ehr_21_23["main_hour"] =  main_ehr_21_23["main_hour"].apply( lambda x: np.sin( (x*np.pi)/24 ) )

main_ehr_18_20.to_csv( loc_ftr + "\\main_ehr_18_20.csv", index = False )
main_ehr_21_23.to_csv( loc_ftr + "\\main_ehr_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Tiempo hasta la primer auction en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer auction

In [27]:
frst_auc_18_20 = rh_18_20.copy()
frst_auc_21_23 = rh_21_23.copy()

first_auctions_rh_ins_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
first_auctions_rh_ins_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date" ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

first_auctions_rh_ins_18_20["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_auctions_rh_ins_21_23["time_to_frt_auc"] = ( pd.to_datetime( first_auctions_rh_ins_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_auctions_rh_ins_18_20.drop( "date", axis = 1, inplace = True )
first_auctions_rh_ins_21_23.drop( "date", axis = 1, inplace = True )

frst_auc_18_20 = frst_auc_18_20.merge( first_auctions_rh_ins_18_20, how = "left", on = "ref_hash" )
frst_auc_21_23 = frst_auc_21_23.merge( first_auctions_rh_ins_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_auc_18_20.to_csv( loc_ftr + "\\frst_auc_18_20.csv", index = False )
frst_auc_21_23.to_csv( loc_ftr + "\\frst_auc_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la primer instlacion en ese ventana** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [ ]:
frst_ins_18_20 = rh_18_20.copy()
frst_ins_21_23 = rh_21_23.copy()

first_installs_rh_auc_18_20 = ins_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_installs_rh_auc_21_23 = ins_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_installs_rh_auc_18_20["time_to_frt_ins"] = ( pd.to_datetime( first_installs_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_installs_rh_auc_21_23["time_to_frt_ins"] = ( pd.to_datetime( first_installs_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_installs_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
first_installs_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

frst_ins_18_20 = frst_ins_18_20.merge( first_installs_rh_auc_18_20, how = "left", on = "ref_hash" )
frst_ins_21_23 = frst_ins_21_23.merge( first_installs_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_ins_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_ins_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_ins_18_20.to_csv( loc_ftr + "\\frst_ins_18_20.csv", index = False )
frst_ins_21_23.to_csv( loc_ftr + "\\frst_ins_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer click** </span> 

Le asignamos cuanto tiempo, en la ventana del 18-20, tardo en realizar su primer instlacion

In [ ]:
frst_clk_18_20 = rh_18_20.copy()
frst_clk_21_23 = rh_21_23.copy()

first_click_rh_auc_18_20 = clk_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_click_rh_auc_21_23 = clk_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_click_rh_auc_18_20["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_click_rh_auc_21_23["time_to_frt_clk"] = ( pd.to_datetime( first_click_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_click_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
first_click_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

frst_clk_18_20 = frst_clk_18_20.merge( first_click_rh_auc_18_20, how = "left", on = "ref_hash" )
frst_clk_21_23 = frst_clk_21_23.merge( first_click_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_clk_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_clk_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_clk_18_20.to_csv( loc_ftr + "\\frst_clk_18_20.csv", index = False )
frst_clk_21_23.to_csv( loc_ftr + "\\frst_clk_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta el primer evento** </span> 

In [ ]:
frst_evt_18_20 = rh_18_20.copy()
frst_evt_21_23 = rh_21_23.copy()

first_event_rh_auc_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )
first_event_rh_auc_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date" ).drop_duplicates( subset = "ref_hash", keep = "first" )

first_event_rh_auc_18_20["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
first_event_rh_auc_21_23["time_to_frt_evt"] = ( pd.to_datetime( first_event_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

first_event_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
first_event_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

frst_evt_18_20 = frst_evt_18_20.merge( first_event_rh_auc_18_20, how = "left", on = "ref_hash" )
frst_evt_21_23 = frst_evt_21_23.merge( first_event_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
frst_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
frst_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

frst_evt_18_20.to_csv( loc_ftr + "\\frst_evt_18_20.csv", index = False )
frst_evt_21_23.to_csv( loc_ftr + "\\frst_evt_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Aplicacion mas instalada por el usuario** </span>

#### <span style="color:orange"> **Mean Encoding** </span> (Usamos el promedio de la cantidad de veces que que es la app principal de algun dispositivo)

In [ ]:
main_app_18_20 = rh_18_20.copy()
main_app_21_23 = rh_21_23.copy()

app_mas_ins_18_20 = ins_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()
app_mas_ins_21_23 = ins_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_ins"} ).reset_index()

app_mas_ins_18_20 = app_mas_ins_18_20.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_18_20["cant_ins"] )
app_mas_ins_21_23 = app_mas_ins_21_23.sort_values( by = ["ref_hash", "cant_ins"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_ins_21_23["cant_ins"] ) 

main_app_18_20 = main_app_18_20.merge( app_mas_ins_18_20, how = "left", on = "ref_hash" )
main_app_21_23 = main_app_21_23.merge( app_mas_ins_21_23, how = "left", on = "ref_hash" )

main_app_18_20["to_count"] = 1; main_app_18_20["application_id"] = main_app_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_18_20); del(main_app_18_20["to_count"])
main_app_21_23["to_count"] = 1; main_app_21_23["application_id"] = main_app_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(main_app_21_23); del(main_app_21_23["to_count"])

main_app_21_23["application_id"].fillna( main_app_21_23["application_id"].isnull().sum() / len(main_app_21_23), inplace = True )
main_app_18_20["application_id"].fillna( main_app_18_20["application_id"].isnull().sum() / len(main_app_18_20), inplace = True )

main_app_18_20.to_csv( loc_ftr + "\\main_app_18_20.csv", index = False )
main_app_21_23.to_csv( loc_ftr + "\\main_app_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones atribuidas por dispositivo** </span> 

In [ ]:
cins_atr_18_20 = rh_18_20.copy()
cins_atr_21_23 = rh_21_23.copy()

cant_atr_ins_18_20 = ins_18_20[["ref_hash", "attributed"]]; cant_atr_ins_18_20["attributed"] = cant_atr_ins_18_20["attributed"].apply( lambda x: 1 if x else 0 );
cant_atr_ins_21_23 = ins_21_23[["ref_hash", "attributed"]]; cant_atr_ins_21_23["attributed"] = cant_atr_ins_21_23["attributed"].apply( lambda x: 1 if x else 0 );

cant_atr_ins_18_20 = cant_atr_ins_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_atr_ins_21_23 = cant_atr_ins_21_23.groupby( "ref_hash" ).agg( "sum" )

cins_atr_18_20 = cins_atr_18_20.merge( cant_atr_ins_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cins_atr_21_23 = cins_atr_21_23.merge( cant_atr_ins_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cins_atr_18_20.to_csv( loc_ftr + "\\cins_atr_18_20.csv", index = False )
cins_atr_21_23.to_csv( loc_ftr + "\\cins_atr_21_23.csv", index = False )

## <span style="color:green"> **Cantidad de instalaciones implicitas por dispositivo** </span> 

In [ ]:
cins_imp_18_20 = rh_18_20.copy()
cins_imp_21_23 = rh_21_23.copy()

cant_imp_ins_18_20 = ins_18_20[["ref_hash", "implicit"]]; cant_imp_ins_18_20["implicit"] = cant_imp_ins_18_20["implicit"].apply( lambda x: 1 if x else 0 );
cant_imp_ins_21_23 = ins_21_23[["ref_hash", "implicit"]]; cant_imp_ins_21_23["implicit"] = cant_imp_ins_21_23["implicit"].apply( lambda x: 1 if x else 0 );

cant_imp_ins_18_20 = cant_imp_ins_18_20.groupby( "ref_hash" ).agg( "sum" )
cant_imp_ins_21_23 = cant_imp_ins_21_23.groupby( "ref_hash" ).agg( "sum" )

cins_imp_18_20 = cins_imp_18_20.merge( cant_imp_ins_18_20, how = "left", on = "ref_hash" ).fillna( 0 )
cins_imp_21_23 = cins_imp_21_23.merge( cant_imp_ins_21_23, how = "left", on = "ref_hash" ).fillna( 0 )

cins_imp_18_20.to_csv( loc_ftr + "\\cins_imp_18_20.csv", index = False )
cins_imp_21_23.to_csv( loc_ftr + "\\cins_imp_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **ID de evento mas realizado por el dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Lo codificamos haciendo la cantidad de eventos que se realizaron de ese id sobre el total de eventos realizados)

In [ ]:
event_id_18_20 = rh_18_20.copy()
event_id_21_23 = rh_21_23.copy()

main_evt_id_18_20 = evt_18_20.groupby( by = ["ref_hash","event_id"] ).agg( {"event_id":"count"} ).rename( columns = {"event_id":"cant_evt"} ).reset_index()
main_evt_id_21_23 = evt_21_23.groupby( by = ["ref_hash","event_id"] ).agg( {"event_id":"count"} ).rename( columns = {"event_id":"cant_evt"} ).reset_index()

main_evt_id_18_20 = main_evt_id_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_evt_id_18_20["cant_evt"] )
main_evt_id_21_23 = main_evt_id_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( main_evt_id_21_23["cant_evt"] ) 

event_id_18_20 = event_id_18_20.merge( main_evt_id_18_20, how = "left", on = "ref_hash" )
event_id_21_23 = event_id_21_23.merge( main_evt_id_21_23, how = "left", on = "ref_hash" )

cant_event_id_18_20 = evt_18_20[["ref_hash","event_id"]].groupby( "event_id" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_id"} )
cant_event_id_21_23 = evt_21_23[["ref_hash","event_id"]].groupby( "event_id" ).agg("count").reset_index().rename( columns = {"ref_hash":"cant_id"} )

event_id_18_20 = event_id_18_20.merge( cant_event_id_18_20, how = "left", on = "event_id" ).drop( "event_id", axis = 1 )
event_id_21_23 = event_id_21_23.merge( cant_event_id_21_23, how = "left", on = "event_id" ).drop( "event_id", axis = 1 )

event_id_18_20["evt_id_mean"] = event_id_18_20["cant_id"] / len(evt_18_20); event_id_18_20.fillna( event_id_18_20["evt_id_mean"].isnull().sum() / len(evt_18_20), inplace = True ); del(event_id_18_20["evt_id_mean"])
event_id_21_23["evt_id_mean"] = event_id_21_23["cant_id"] / len(evt_21_23); event_id_21_23.fillna( event_id_21_23["evt_id_mean"].isnull().sum() / len(evt_21_23), inplace = True ); del(event_id_21_23["evt_id_mean"])

event_id_18_20.to_csv( loc_ftr + "\\event_id_18_20.csv", index = False )
event_id_21_23.to_csv( loc_ftr + "\\event_id_21_23.csv", index = False )

## <span style="color:green"> **Aplicacion que mas recibio eventos por cada dispositivo** </span> 

#### <span style="color:orange"> **Mean Encoding** </span> (Codificiamos sobre cuantas veces la aplicacion fue la principal de un dispositivo sobre el total de dispositivos)

In [ ]:
mapp_evt_18_20 = rh_18_20.copy()
mapp_evt_21_23 = rh_21_23.copy()

app_mas_evt_18_20 = evt_18_20.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()
app_mas_evt_21_23 = evt_21_23.groupby( by = ["ref_hash","application_id"] ).agg( {"application_id":"count"} ).rename( columns = {"application_id":"cant_evt"} ).reset_index()

app_mas_evt_18_20 = app_mas_evt_18_20.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_18_20["cant_evt"] )
app_mas_evt_21_23 = app_mas_evt_21_23.sort_values( by = ["ref_hash", "cant_evt"], ascending = True ).drop_duplicates( subset = "ref_hash", keep = "last" ); del( app_mas_evt_21_23["cant_evt"] ) 

mapp_evt_18_20 = mapp_evt_18_20.merge( app_mas_evt_18_20, how = "left", on = "ref_hash" )
mapp_evt_21_23 = mapp_evt_21_23.merge( app_mas_evt_21_23, how = "left", on = "ref_hash" )

mapp_evt_18_20["to_count"] = 1; mapp_evt_18_20["application_id"] = mapp_evt_18_20[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_18_20); del(mapp_evt_18_20["to_count"])
mapp_evt_21_23["to_count"] = 1; mapp_evt_21_23["application_id"] = mapp_evt_21_23[["application_id", "to_count"]].groupby("application_id").transform( "sum" ) / len(mapp_evt_21_23); del(mapp_evt_21_23["to_count"])

#Completamos los NaNs con el promedio de su cantidad
mapp_evt_18_20["application_id"].fillna( mapp_evt_18_20["application_id"].isnull().sum() / len( mapp_evt_18_20 ) , inplace = True )
mapp_evt_21_23["application_id"].fillna( mapp_evt_21_23["application_id"].isnull().sum() / len( mapp_evt_21_23 ), inplace = True )

mapp_evt_18_20.to_csv( loc_ftr + "\\mapp_evt_18_20.csv", index = False )
mapp_evt_21_23.to_csv( loc_ftr + "\\mapp_evt_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Tiempo hasta la ultima auction en esa ventana** </span> 

In [22]:
last_auc_18_20 = rh_18_20.copy()
last_auc_21_23 = rh_21_23.copy()

last_auctions_rh_auc_18_20 = auc_18_20[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )
last_auctions_rh_auc_21_23 = auc_21_23[ ["device_id","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "device_id", keep = "first" ).rename( columns = {"device_id":"ref_hash"} )

last_auctions_rh_auc_18_20["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_auctions_rh_auc_21_23["time_to_lst_auc"] = ( pd.to_datetime( last_auctions_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_auctions_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
last_auctions_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

last_auc_18_20 = last_auc_18_20.merge( last_auctions_rh_auc_18_20, how = "left", on = "ref_hash" )
last_auc_21_23 = last_auc_21_23.merge( last_auctions_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_auc_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_auc_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_auc_18_20.to_csv( loc_ftr + "\\last_auc_18_20.csv", index = False )
last_auc_21_23.to_csv( loc_ftr + "\\last_auc_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultimo evento en esa ventana** </span> 

In [23]:
last_evt_18_20 = rh_18_20.copy()
last_evt_21_23 = rh_21_23.copy()

last_event_rh_auc_18_20 = evt_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_event_rh_auc_21_23 = evt_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_event_rh_auc_18_20["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_event_rh_auc_21_23["time_to_lst_evt"] = ( pd.to_datetime( last_event_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_event_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
last_event_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

last_evt_18_20 = last_evt_18_20.merge( last_event_rh_auc_18_20, how = "left", on = "ref_hash" )
last_evt_21_23 = last_evt_21_23.merge( last_event_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_evt_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_evt_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_evt_18_20.to_csv( loc_ftr + "\\last_evt_18_20.csv", index = False )
last_evt_21_23.to_csv( loc_ftr + "\\last_evt_21_23.csv", index = False )

## <span style="color:green"> **Tiempo hasta la ultima instalacion en esa ventana** </span>

In [24]:
last_ins_18_20 = rh_18_20.copy()
last_ins_21_23 = rh_21_23.copy()

last_installs_rh_auc_18_20 = ins_18_20[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )
last_installs_rh_auc_21_23 = ins_21_23[ ["ref_hash","date"] ].sort_values( "date", ascending = False ).drop_duplicates( subset = "ref_hash", keep = "first" )

last_installs_rh_auc_18_20["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_auc_18_20["date"] ) -  dt.datetime( year = 2019, month = 4, day = 18 ) ).dt.total_seconds()
last_installs_rh_auc_21_23["time_to_lst_ins"] = ( pd.to_datetime( last_installs_rh_auc_21_23["date"] ) -  dt.datetime( year = 2019, month = 4, day = 21 ) ).dt.total_seconds()

last_installs_rh_auc_18_20.drop( "date", axis = 1, inplace = True )
last_installs_rh_auc_21_23.drop( "date", axis = 1, inplace = True )

last_ins_18_20 = last_ins_18_20.merge( last_installs_rh_auc_18_20, how = "left", on = "ref_hash" )
last_ins_21_23 = last_ins_21_23.merge( last_installs_rh_auc_21_23, how = "left", on = "ref_hash" )

#Los que tienen NaN es que nunca convirtieron. Los marcamos con el tiempo maximo
last_ins_18_20.fillna( 3 * 24 * 3600, inplace = True )
last_ins_21_23.fillna( 3 * 24 * 3600, inplace = True )

last_ins_18_20.to_csv( loc_ftr + "\\last_ins_18_20.csv", index = False )
last_ins_21_23.to_csv( loc_ftr + "\\last_ins_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Participo en mas de una subasta** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20['mt_1_auc'] = (cant_auc_18_20['mt_1_auc'] > 1).astype('int8')
cant_auc_21_23['mt_1_auc'] = (cant_auc_21_23['mt_1_auc'] > 1).astype('int8')

cant_auc_18_20['me_mt_1_auc'] = ( cant_auc_18_20.groupby('mt_1_auc').transform('count') ) / len(cant_auc_18_20['mt_1_auc'])
cant_auc_21_23['me_mt_1_auc'] = ( cant_auc_21_23.groupby('mt_1_auc').transform('count') ) / len(cant_auc_21_23['mt_1_auc'])

del cant_auc_18_20['mt_1_auc']
del cant_auc_21_23['mt_1_auc']

cant_auc_18_20.to_csv( loc_ftr + "\\me_mt_1_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\me_mt_1_auc_21_23.csv", index = False )

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [29]:
cant_auc_18_20 = rh_18_20.copy()
cant_auc_21_23 = rh_21_23.copy()

cant_auc_18_20 = cant_auc_18_20.merge( auc_18_20["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )
cant_auc_21_23 = cant_auc_21_23.merge( auc_21_23["device_id"].value_counts().to_frame().reset_index().rename( columns = {"device_id": "mt_1_auc", "index":"ref_hash"} ), how = "left", on = "ref_hash" )

cant_auc_18_20.fillna( 0, inplace = True )
cant_auc_21_23.fillna( 0, inplace = True )

cant_auc_18_20['mt_1_auc'] = (cant_auc_18_20['mt_1_auc'] > 1).astype('int8')
cant_auc_21_23['mt_1_auc'] = (cant_auc_21_23['mt_1_auc'] > 1).astype('int8')

cant_auc_18_20.to_csv( loc_ftr + "\\mt_1_auc_18_20.csv", index = False )
cant_auc_21_23.to_csv( loc_ftr + "\\mt_1_auc_21_23.csv", index = False )

## =========================================================================

## <span style="color:green"> **Recibio subasta entre 21 y 3** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
auc_18_20['auc_21_3'] = (auc_18_20['date'].dt.hour < 4) | (auc_18_20['date'].dt.hour > 20)
auc_21_23['auc_21_3'] = (auc_21_23['date'].dt.hour < 4) | (auc_21_23['date'].dt.hour > 20)

auc_night_18_20 = rh_18_20.copy()
auc_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20['me_auc_21_3'] = ( auc_night_18_20.groupby('auc_21_3').transform('count') ) / len(auc_night_18_20['auc_21_3'])
auc_night_21_23['me_auc_21_3'] = ( auc_night_21_23.groupby('auc_21_3').transform('count') ) / len(auc_night_21_23['auc_21_3'])

del auc_night_18_20['auc_21_3']
del auc_night_21_23['auc_21_3']

auc_night_18_20.to_csv( loc_ftr + "\\me_auc_night_18_20.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\me_auc_night_21_23.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [38]:
auc_18_20['auc_21_3'] = (auc_18_20['date'].dt.hour < 4) | (auc_18_20['date'].dt.hour > 20)
auc_21_23['auc_21_3'] = (auc_21_23['date'].dt.hour < 4) | (auc_21_23['date'].dt.hour > 20)

auc_night_18_20 = rh_18_20.copy()
auc_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_21_3':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_night_18_20 = auc_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_night_21_23 = auc_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_night_18_20["auc_21_3"] =  (auc_night_18_20["auc_21_3"] > 1).astype('int8')
auc_night_21_23["auc_21_3"] =  (auc_night_21_23["auc_21_3"] > 1).astype('int8')

auc_night_18_20.to_csv( loc_ftr + "\\auc_night_18_20.csv", index = False )
auc_night_21_23.to_csv( loc_ftr + "\\auc_night_21_23.csv", index = False )

del auc_18_20['auc_21_3']
del auc_21_23['auc_21_3']

## <span style="color:green"> **Recibio subasta entre 4 y 10** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
auc_18_20['auc_4_10'] = (auc_18_20['date'].dt.hour > 3) & (auc_18_20['date'].dt.hour < 11) 
auc_21_23['auc_4_10'] = (auc_21_23['date'].dt.hour > 3) & (auc_21_23['date'].dt.hour < 11)

auc_morn_18_20 = rh_18_20.copy()
auc_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_18_20 = auc_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_morn_21_23 = auc_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_morn_18_20["auc_4_10"] =  (auc_morn_18_20["auc_4_10"] > 1).astype('int8')
auc_morn_21_23["auc_4_10"] =  (auc_morn_21_23["auc_4_10"] > 1).astype('int8')

auc_morn_18_20['me_auc_4_10'] = ( auc_morn_18_20.groupby('auc_4_10').transform('count') ) / len(auc_morn_18_20['auc_4_10'])
auc_morn_21_23['me_auc_4_10'] = ( auc_morn_21_23.groupby('auc_4_10').transform('count') ) / len(auc_morn_21_23['auc_4_10'])

del auc_morn_18_20['auc_4_10']
del auc_morn_21_23['auc_4_10']

auc_morn_18_20.to_csv( loc_ftr + "\\me_auc_morn_18_20.csv", index = False )
auc_morn_21_23.to_csv( loc_ftr + "\\me_auc_morn_21_23.csv", index = False )

del auc_18_20['auc_4_10']
del auc_21_23['auc_4_10']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [42]:
auc_18_20['auc_4_10'] = (auc_18_20['date'].dt.hour > 3) & (auc_18_20['date'].dt.hour < 11) 
auc_21_23['auc_4_10'] = (auc_21_23['date'].dt.hour > 3) & (auc_21_23['date'].dt.hour < 11)

auc_morn_18_20 = rh_18_20.copy()
auc_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_4_10':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_morn_18_20 = auc_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_morn_21_23 = auc_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_morn_18_20["auc_4_10"] =  (auc_morn_18_20["auc_4_10"] > 1).astype('int8')
auc_morn_21_23["auc_4_10"] =  (auc_morn_21_23["auc_4_10"] > 1).astype('int8')

auc_morn_18_20.to_csv( loc_ftr + "\\auc_morn_18_20.csv", index = False )
auc_morn_21_23.to_csv( loc_ftr + "\\auc_morn_21_23.csv", index = False )

del auc_18_20['auc_4_10']
del auc_21_23['auc_4_10']

## <span style="color:green"> **Recibio subasta entre 11 y 15** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
auc_18_20['auc_11_15'] = (auc_18_20['date'].dt.hour > 10) & (auc_18_20['date'].dt.hour < 16) 
auc_21_23['auc_11_15'] = (auc_21_23['date'].dt.hour > 10) & (auc_21_23['date'].dt.hour < 16)

auc_midday_18_20 = rh_18_20.copy()
auc_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_18_20 = auc_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_midday_21_23 = auc_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_midday_18_20["auc_11_15"] =  (auc_midday_18_20["auc_11_15"] > 1).astype('int8')
auc_midday_21_23["auc_11_15"] =  (auc_midday_21_23["auc_11_15"] > 1).astype('int8')

auc_midday_18_20['me_auc_11_15'] = ( auc_midday_18_20.groupby('auc_11_15').transform('count') ) / len(auc_midday_18_20['auc_11_15'])
auc_midday_21_23['me_auc_11_15'] = ( auc_midday_21_23.groupby('auc_11_15').transform('count') ) / len(auc_midday_21_23['auc_11_15'])

del auc_midday_18_20['auc_11_15']
del auc_midday_21_23['auc_11_15']

auc_midday_18_20.to_csv( loc_ftr + "\\me_auc_midday_18_20.csv", index = False )
auc_midday_21_23.to_csv( loc_ftr + "\\me_auc_midday_21_23.csv", index = False )

del auc_18_20['auc_11_15']
del auc_21_23['auc_11_15']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [44]:
auc_18_20['auc_11_15'] = (auc_18_20['date'].dt.hour > 10) & (auc_18_20['date'].dt.hour < 16) 
auc_21_23['auc_11_15'] = (auc_21_23['date'].dt.hour > 10) & (auc_21_23['date'].dt.hour < 16)

auc_midday_18_20 = rh_18_20.copy()
auc_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_11_15':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_midday_18_20 = auc_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_midday_21_23 = auc_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_midday_18_20["auc_11_15"] =  (auc_midday_18_20["auc_11_15"] > 1).astype('int8')
auc_midday_21_23["auc_11_15"] =  (auc_midday_21_23["auc_11_15"] > 1).astype('int8')

auc_midday_18_20.to_csv( loc_ftr + "\\auc_midday_18_20.csv", index = False )
auc_midday_21_23.to_csv( loc_ftr + "\\auc_midday_21_23.csv", index = False )

del auc_18_20['auc_11_15']
del auc_21_23['auc_11_15']

## <span style="color:green"> **Recibio subasta entre 16 y 20** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
auc_18_20['auc_16_20'] = (auc_18_20['date'].dt.hour > 15) & (auc_18_20['date'].dt.hour < 21) 
auc_21_23['auc_16_20'] = (auc_21_23['date'].dt.hour > 15) & (auc_21_23['date'].dt.hour < 21)

auc_after_18_20 = rh_18_20.copy()
auc_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_18_20 = auc_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_after_21_23 = auc_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_after_18_20["auc_16_20"] =  (auc_after_18_20["auc_16_20"] > 1).astype('int8')
auc_after_21_23["auc_16_20"] =  (auc_after_21_23["auc_16_20"] > 1).astype('int8')

auc_after_18_20['me_auc_16_20'] = ( auc_after_18_20.groupby('auc_16_20').transform('count') ) / len(auc_after_18_20['auc_16_20'])
auc_after_21_23['me_auc_16_20'] = ( auc_after_21_23.groupby('auc_16_20').transform('count') ) / len(auc_after_21_23['auc_16_20'])

del auc_after_18_20['auc_16_20']
del auc_after_21_23['auc_16_20']

auc_after_18_20.to_csv( loc_ftr + "\\me_auc_after_18_20.csv", index = False )
auc_after_21_23.to_csv( loc_ftr + "\\me_auc_after_21_23.csv", index = False )

del auc_18_20['auc_16_20']
del auc_21_23['auc_16_20']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [46]:
auc_18_20['auc_16_20'] = (auc_18_20['date'].dt.hour > 15) & (auc_18_20['date'].dt.hour < 21) 
auc_21_23['auc_16_20'] = (auc_21_23['date'].dt.hour > 15) & (auc_21_23['date'].dt.hour < 21)

auc_after_18_20 = rh_18_20.copy()
auc_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = auc_18_20.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )
hour_mode_21_23 = auc_21_23.groupby('device_id').agg({'auc_16_20':'sum'}).reset_index().rename( columns = {"device_id":"ref_hash"} )

auc_after_18_20 = auc_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
auc_after_21_23 = auc_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

auc_after_18_20["auc_16_20"] =  (auc_after_18_20["auc_16_20"] > 1).astype('int8')
auc_after_21_23["auc_16_20"] =  (auc_after_21_23["auc_16_20"] > 1).astype('int8')

auc_after_18_20.to_csv( loc_ftr + "\\auc_after_18_20.csv", index = False )
auc_after_21_23.to_csv( loc_ftr + "\\auc_after_21_23.csv", index = False )

del auc_18_20['auc_16_20']
del auc_21_23['auc_16_20']

## =========================================================================

## <span style="color:green"> **Hizo eventos entre 21 hs y 3 hs (Noche)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_21_3'] = (evt_18_20['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)
evt_21_23['evt_21_3'] = (evt_21_23['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)

evt_night_18_20 = rh_18_20.copy()
evt_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_18_20 = evt_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_night_21_23 = evt_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_night_18_20["evt_21_3"] =  (evt_night_18_20["evt_21_3"] > 1).astype('int8')
evt_night_21_23["evt_21_3"] =  (evt_night_21_23["evt_21_3"] > 1).astype('int8')

evt_night_18_20['me_evt_21_3'] = ( evt_night_18_20.groupby('evt_21_3').transform('count') ) / len(evt_night_18_20['evt_21_3'])
evt_night_21_23['me_evt_21_3'] = ( evt_night_21_23.groupby('evt_21_3').transform('count') ) / len(evt_night_21_23['evt_21_3'])

del evt_night_18_20['evt_21_3']
del evt_night_21_23['evt_21_3']

evt_night_18_20.to_csv( loc_ftr + "\\me_evt_night_18_20.csv", index = False )
evt_night_21_23.to_csv( loc_ftr + "\\me_evt_night_21_23.csv", index = False )

del evt_18_20['evt_21_3']
del evt_21_23['evt_21_3']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [51]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_21_3'] = (evt_18_20['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)
evt_21_23['evt_21_3'] = (evt_21_23['date'].dt.hour < 4) | (evt_18_20['date'].dt.hour > 20)

evt_night_18_20 = rh_18_20.copy()
evt_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_21_3':'sum'}).reset_index()

evt_night_18_20 = evt_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_night_21_23 = evt_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_night_18_20["evt_21_3"] =  (evt_night_18_20["evt_21_3"] > 1).astype('int8')
evt_night_21_23["evt_21_3"] =  (evt_night_21_23["evt_21_3"] > 1).astype('int8')

evt_night_18_20.to_csv( loc_ftr + "\\evt_night_18_20.csv", index = False )
evt_night_21_23.to_csv( loc_ftr + "\\evt_night_21_23.csv", index = False )

del evt_18_20['evt_21_3']
del evt_21_23['evt_21_3']

## <span style="color:green"> **Hizo eventos entre 4 hs y 10 hs (Mañana)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_4_10'] = (evt_18_20['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)
evt_21_23['evt_4_10'] = (evt_21_23['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)

evt_morn_18_20 = rh_18_20.copy()
evt_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_18_20 = evt_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_morn_21_23 = evt_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_morn_18_20["evt_4_10"] =  (evt_morn_18_20["evt_4_10"] > 1).astype('int8')
evt_morn_21_23["evt_4_10"] =  (evt_morn_21_23["evt_4_10"] > 1).astype('int8')

evt_morn_18_20['me_evt_4_10'] = ( evt_morn_18_20.groupby('evt_4_10').transform('count') ) / len(evt_morn_18_20['evt_4_10'])
evt_morn_21_23['me_evt_4_10'] = ( evt_morn_21_23.groupby('evt_4_10').transform('count') ) / len(evt_morn_21_23['evt_4_10'])

del evt_morn_18_20['evt_4_10']
del evt_morn_21_23['evt_4_10']

evt_morn_18_20.to_csv( loc_ftr + "\\me_evt_morn_18_20.csv", index = False )
evt_morn_21_23.to_csv( loc_ftr + "\\me_evt_morn_21_23.csv", index = False )

del evt_18_20['evt_4_10']
del evt_21_23['evt_4_10']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [51]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_4_10'] = (evt_18_20['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)
evt_21_23['evt_4_10'] = (evt_21_23['date'].dt.hour < 11) & (evt_18_20['date'].dt.hour > 3)

evt_morn_18_20 = rh_18_20.copy()
evt_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_4_10':'sum'}).reset_index()

evt_morn_18_20 = evt_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_morn_21_23 = evt_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_morn_18_20["evt_4_10"] =  (evt_morn_18_20["evt_4_10"] > 1).astype('int8')
evt_morn_21_23["evt_4_10"] =  (evt_morn_21_23["evt_4_10"] > 1).astype('int8')

evt_morn_18_20.to_csv( loc_ftr + "\\evt_morn_18_20.csv", index = False )
evt_morn_21_23.to_csv( loc_ftr + "\\evt_morn_21_23.csv", index = False )

del evt_18_20['evt_4_10']
del evt_21_23['evt_4_10']

## <span style="color:green"> **Hizo eventos entre 11 hs y 15 hs (Mediodia)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_11_15'] = (evt_18_20['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)
evt_21_23['evt_11_15'] = (evt_21_23['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)

evt_midday_18_20 = rh_18_20.copy()
evt_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_18_20 = evt_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_midday_21_23 = evt_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_midday_18_20["evt_11_15"] =  (evt_midday_18_20["evt_11_15"] > 1).astype('int8')
evt_midday_21_23["evt_11_15"] =  (evt_midday_21_23["evt_11_15"] > 1).astype('int8')

evt_midday_18_20['me_evt_11_15'] = ( evt_midday_18_20.groupby('evt_11_15').transform('count') ) / len(evt_midday_18_20['evt_11_15'])
evt_midday_21_23['me_evt_11_15'] = ( evt_midday_21_23.groupby('evt_11_15').transform('count') ) / len(evt_midday_21_23['evt_11_15'])

del evt_midday_18_20['evt_11_15']
del evt_midday_21_23['evt_11_15']

evt_midday_18_20.to_csv( loc_ftr + "\\me_evt_midday_18_20.csv", index = False )
evt_midday_21_23.to_csv( loc_ftr + "\\me_evt_midday_21_23.csv", index = False )

del evt_18_20['evt_11_15']
del evt_21_23['evt_11_15']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [51]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_11_15'] = (evt_18_20['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)
evt_21_23['evt_11_15'] = (evt_21_23['date'].dt.hour < 16) & (evt_18_20['date'].dt.hour > 10)

evt_midday_18_20 = rh_18_20.copy()
evt_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_11_15':'sum'}).reset_index()

evt_midday_18_20 = evt_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_midday_21_23 = evt_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_midday_18_20["evt_11_15"] =  (evt_midday_18_20["evt_11_15"] > 1).astype('int8')
evt_midday_21_23["evt_11_15"] =  (evt_midday_21_23["evt_11_15"] > 1).astype('int8')

evt_midday_18_20.to_csv( loc_ftr + "\\evt_midday_18_20.csv", index = False )
evt_midday_21_23.to_csv( loc_ftr + "\\evt_midday_21_23.csv", index = False )

del evt_18_20['evt_11_15']
del evt_21_23['evt_11_15']

## <span style="color:green"> **Hizo eventos entre 16 hs y 20 hs (Tarde)** </span>

## <span style="color:Orange"> **Mean Encoding** </span> 

In [ ]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_16_20'] = (evt_18_20['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)
evt_21_23['evt_16_20'] = (evt_21_23['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)

evt_after_18_20 = rh_18_20.copy()
evt_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_18_20 = evt_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_after_21_23 = evt_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_after_18_20["evt_16_20"] =  (evt_after_18_20["evt_16_20"] > 1).astype('int8')
evt_after_21_23["evt_16_20"] =  (evt_after_21_23["evt_16_20"] > 1).astype('int8')

evt_after_18_20['me_evt_16_20'] = ( evt_after_18_20.groupby('evt_16_20').transform('count') ) / len(evt_after_18_20['evt_16_20'])
evt_after_21_23['me_evt_16_20'] = ( evt_after_21_23.groupby('evt_16_20').transform('count') ) / len(evt_after_21_23['evt_16_20'])

del evt_after_18_20['evt_16_20']
del evt_after_21_23['evt_16_20']

evt_after_18_20.to_csv( loc_ftr + "\\me_evt_after_18_20.csv", index = False )
evt_after_21_23.to_csv( loc_ftr + "\\me_evt_after_21_23.csv", index = False )

del evt_18_20['evt_16_20']
del evt_21_23['evt_16_20']

## <span style="color:Orange"> **One-Hot Encoding** </span> 

In [51]:
evt_18_20['date'] = pd.to_datetime(evt_18_20['date'])
evt_21_23['date'] = pd.to_datetime(evt_21_23['date'])

evt_18_20['evt_16_20'] = (evt_18_20['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)
evt_21_23['evt_16_20'] = (evt_21_23['date'].dt.hour < 21) & (evt_18_20['date'].dt.hour > 15)

evt_after_18_20 = rh_18_20.copy()
evt_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = evt_18_20.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()
hour_mode_21_23 = evt_21_23.groupby('ref_hash').agg({'evt_16_20':'sum'}).reset_index()

evt_after_18_20 = evt_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
evt_after_21_23 = evt_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

evt_after_18_20["evt_16_20"] =  (evt_after_18_20["evt_16_20"] > 1).astype('int8')
evt_after_21_23["evt_16_20"] =  (evt_after_21_23["evt_16_20"] > 1).astype('int8')

evt_after_18_20.to_csv( loc_ftr + "\\evt_after_18_20.csv", index = False )
evt_after_21_23.to_csv( loc_ftr + "\\evt_after_21_23.csv", index = False )

del evt_18_20['evt_16_20']
del evt_21_23['evt_16_20']

## =========================================================================

## <span style="color:green"> **Instalo entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [23]:
ins_18_20['ins_21_3'] = (ins_18_20['date'].dt.hour > 20) | (ins_18_20['date'].dt.hour < 4) 
ins_21_23['ins_21_3'] = (ins_21_23['date'].dt.hour > 20) | (ins_21_23['date'].dt.hour < 4)

ins_night_18_20 = rh_18_20.copy()
ins_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_18_20 = ins_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_night_21_23 = ins_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_night_18_20["ins_21_3"] =  (ins_night_18_20["ins_21_3"] > 1).astype('int8')
ins_night_21_23["ins_21_3"] =  (ins_night_21_23["ins_21_3"] > 1).astype('int8')

ins_night_18_20['me_ins_21_3'] = ( ins_night_18_20.groupby('ins_21_3').transform('count') ) / len(ins_night_18_20['ins_21_3'])
ins_night_21_23['me_ins_21_3'] = ( ins_night_21_23.groupby('ins_21_3').transform('count') ) / len(ins_night_21_23['ins_21_3'])

del ins_night_18_20["ins_21_3"]
del ins_night_21_23["ins_21_3"]

ins_night_18_20.to_csv( loc_ftr + "\\me_ins_night_18_20.csv", index = False )
ins_night_21_23.to_csv( loc_ftr + "\\me_ins_night_21_23.csv", index = False )

del ins_18_20['ins_21_3']
del ins_21_23['ins_21_3']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [24]:
ins_18_20['ins_21_3'] = (ins_18_20['date'].dt.hour > 20) | (ins_18_20['date'].dt.hour < 4) 
ins_21_23['ins_21_3'] = (ins_21_23['date'].dt.hour > 20) | (ins_21_23['date'].dt.hour < 4)

ins_night_18_20 = rh_18_20.copy()
ins_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_21_3':'sum'}).reset_index()

ins_night_18_20 = ins_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_night_21_23 = ins_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_night_18_20["ins_21_3"] =  (ins_night_18_20["ins_21_3"] > 1).astype('int8')
ins_night_21_23["ins_21_3"] =  (ins_night_21_23["ins_21_3"] > 1).astype('int8')

ins_night_18_20.to_csv( loc_ftr + "\\ins_night_18_20.csv", index = False )
ins_night_21_23.to_csv( loc_ftr + "\\ins_night_21_23.csv", index = False )

del ins_18_20['ins_21_3']
del ins_21_23['ins_21_3']

## <span style="color:green"> **Instalo entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [25]:
ins_18_20['ins_4_10'] = (ins_18_20['date'].dt.hour > 3) & (ins_18_20['date'].dt.hour < 11) 
ins_21_23['ins_4_10'] = (ins_21_23['date'].dt.hour > 3) & (ins_21_23['date'].dt.hour < 11)

ins_morn_18_20 = rh_18_20.copy()
ins_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_18_20 = ins_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_morn_21_23 = ins_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_morn_18_20["ins_4_10"] =  (ins_morn_18_20["ins_4_10"] > 1).astype('int8')
ins_morn_21_23["ins_4_10"] =  (ins_morn_21_23["ins_4_10"] > 1).astype('int8')

ins_morn_18_20['me_ins_4_10'] = ( ins_morn_18_20.groupby('ins_4_10').transform('count') ) / len(ins_morn_18_20['ins_4_10'])
ins_morn_21_23['me_ins_4_10'] = ( ins_morn_21_23.groupby('ins_4_10').transform('count') ) / len(ins_morn_21_23['ins_4_10'])

del ins_morn_18_20["ins_4_10"]
del ins_morn_21_23["ins_4_10"]

ins_morn_18_20.to_csv( loc_ftr + "\\me_ins_morn_18_20.csv", index = False )
ins_morn_21_23.to_csv( loc_ftr + "\\me_ins_morn_21_23.csv", index = False )

del ins_18_20['ins_4_10']
del ins_21_23['ins_4_10']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [26]:
ins_18_20['ins_4_10'] = (ins_18_20['date'].dt.hour > 3) & (ins_18_20['date'].dt.hour < 11) 
ins_21_23['ins_4_10'] = (ins_21_23['date'].dt.hour > 3) & (ins_21_23['date'].dt.hour < 11)

ins_morn_18_20 = rh_18_20.copy()
ins_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_4_10':'sum'}).reset_index()

ins_morn_18_20 = ins_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_morn_21_23 = ins_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_morn_18_20["ins_4_10"] =  (ins_morn_18_20["ins_4_10"] > 1).astype('int8')
ins_morn_21_23["ins_4_10"] =  (ins_morn_21_23["ins_4_10"] > 1).astype('int8')

ins_morn_18_20.to_csv( loc_ftr + "\\ins_morn_18_20.csv", index = False )
ins_morn_21_23.to_csv( loc_ftr + "\\ins_morn_21_23.csv", index = False )

del ins_18_20['ins_4_10']
del ins_21_23['ins_4_10']

## <span style="color:green"> **Instalo entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [27]:
ins_18_20['ins_11_15'] = (ins_18_20['date'].dt.hour > 10) & (ins_18_20['date'].dt.hour < 16) 
ins_21_23['ins_11_15'] = (ins_21_23['date'].dt.hour > 10) & (ins_21_23['date'].dt.hour < 16)

ins_midday_18_20 = rh_18_20.copy()
ins_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_18_20 = ins_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_midday_21_23 = ins_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_midday_18_20["ins_11_15"] =  (ins_midday_18_20["ins_11_15"] > 1).astype('int8')
ins_midday_21_23["ins_11_15"] =  (ins_midday_21_23["ins_11_15"] > 1).astype('int8')

ins_midday_18_20['me_ins_11_15'] = ( ins_midday_18_20.groupby('ins_11_15').transform('count') ) / len(ins_midday_18_20['ins_11_15'])
ins_midday_21_23['me_ins_11_15'] = ( ins_midday_21_23.groupby('ins_11_15').transform('count') ) / len(ins_midday_21_23['ins_11_15'])

del ins_midday_18_20["ins_11_15"]
del ins_midday_21_23["ins_11_15"]

ins_midday_18_20.to_csv( loc_ftr + "\\me_ins_midday_18_20.csv", index = False )
ins_midday_21_23.to_csv( loc_ftr + "\\me_ins_midday_21_23.csv", index = False )

del ins_18_20['ins_11_15']
del ins_21_23['ins_11_15']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [28]:
ins_18_20['ins_11_15'] = (ins_18_20['date'].dt.hour > 10) & (ins_18_20['date'].dt.hour < 16) 
ins_21_23['ins_11_15'] = (ins_21_23['date'].dt.hour > 10) & (ins_21_23['date'].dt.hour < 16)

ins_midday_18_20 = rh_18_20.copy()
ins_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_11_15':'sum'}).reset_index()

ins_midday_18_20 = ins_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_midday_21_23 = ins_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_midday_18_20["ins_11_15"] =  (ins_midday_18_20["ins_11_15"] > 1).astype('int8')
ins_midday_21_23["ins_11_15"] =  (ins_midday_21_23["ins_11_15"] > 1).astype('int8')

ins_midday_18_20.to_csv( loc_ftr + "\\ins_midday_18_20.csv", index = False )
ins_midday_21_23.to_csv( loc_ftr + "\\ins_midday_21_23.csv", index = False )

del ins_18_20['ins_11_15']
del ins_21_23['ins_11_15']

## <span style="color:green"> **Instalo entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
ins_18_20['ins_16_20'] = (ins_18_20['date'].dt.hour > 15) & (ins_18_20['date'].dt.hour < 21) 
ins_21_23['ins_16_20'] = (ins_21_23['date'].dt.hour > 15) & (ins_21_23['date'].dt.hour < 21)

ins_after_18_20 = rh_18_20.copy()
ins_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_18_20 = ins_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_after_21_23 = ins_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_after_18_20["ins_16_20"] =  (ins_after_18_20["ins_16_20"] > 1).astype('int8')
ins_after_21_23["ins_16_20"] =  (ins_after_21_23["ins_16_20"] > 1).astype('int8')

ins_after_18_20['me_ins_16_20'] = ( ins_after_18_20.groupby('ins_16_20').transform('count') ) / len(ins_after_18_20['ins_16_20'])
ins_after_21_23['me_ins_16_20'] = ( ins_after_21_23.groupby('ins_16_20').transform('count') ) / len(ins_after_21_23['ins_16_20'])

del ins_after_18_20["ins_16_20"]
del ins_after_21_23["ins_16_20"]


ins_after_18_20.to_csv( loc_ftr + "\\me_ins_after_18_20.csv", index = False )
ins_after_21_23.to_csv( loc_ftr + "\\me_ins_after_21_23.csv", index = False )

del ins_18_20['ins_16_20']
del ins_21_23['ins_16_20']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [30]:
ins_18_20['ins_16_20'] = (ins_18_20['date'].dt.hour > 15) & (ins_18_20['date'].dt.hour < 21) 
ins_21_23['ins_16_20'] = (ins_21_23['date'].dt.hour > 15) & (ins_21_23['date'].dt.hour < 21)

ins_after_18_20 = rh_18_20.copy()
ins_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = ins_18_20.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()
hour_mode_21_23 = ins_21_23.groupby('ref_hash').agg({'ins_16_20':'sum'}).reset_index()

ins_after_18_20 = ins_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
ins_after_21_23 = ins_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

ins_after_18_20["ins_16_20"] =  (ins_after_18_20["ins_16_20"] > 1).astype('int8')
ins_after_21_23["ins_16_20"] =  (ins_after_21_23["ins_16_20"] > 1).astype('int8')

ins_after_18_20.to_csv( loc_ftr + "\\ins_after_18_20.csv", index = False )
ins_after_21_23.to_csv( loc_ftr + "\\ins_after_21_23.csv", index = False )

del ins_18_20['ins_16_20']
del ins_21_23['ins_16_20']

## =========================================================================

## <span style="color:green"> **Recibio clicks entre 16 hs y 20 hs (Tarde)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
clk_18_20['clk_16_20'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 21) 
clk_21_23['clk_16_20'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 21)

clk_after_18_20 = rh_18_20.copy()
clk_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_18_20 = clk_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_after_21_23 = clk_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_after_18_20["clk_16_20"] = (clk_after_18_20["clk_16_20"] > 1).astype('int8')
clk_after_21_23["clk_16_20"] = (clk_after_21_23["clk_16_20"] > 1).astype('int8')

clk_after_18_20['me_clk_16_20'] = ( clk_after_18_20.groupby('clk_16_20').transform('count') ) / len(clk_after_18_20['clk_16_20'])
clk_after_21_23['me_clk_16_20'] = ( clk_after_21_23.groupby('clk_16_20').transform('count') ) / len(clk_after_21_23['clk_16_20'])

del clk_after_18_20["clk_16_20"]
del clk_after_21_23["clk_16_20"]

clk_after_18_20.to_csv( loc_ftr + "\\me_clk_after_18_20.csv", index = False )
clk_after_21_23.to_csv( loc_ftr + "\\me_clk_after_21_23.csv", index = False )

del clk_18_20['clk_16_20']
del clk_21_23['clk_16_20']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [35]:
clk_18_20['clk_16_20'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 21) 
clk_21_23['clk_16_20'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 15) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 21)

clk_after_18_20 = rh_18_20.copy()
clk_after_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_16_20':'sum'}).reset_index()

clk_after_18_20 = clk_after_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_after_21_23 = clk_after_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_after_18_20["clk_16_20"] = (clk_after_18_20["clk_16_20"] > 1).astype('int8')
clk_after_21_23["clk_16_20"] = (clk_after_21_23["clk_16_20"] > 1).astype('int8')

clk_after_18_20.to_csv( loc_ftr + "\\clk_after_18_20.csv", index = False )
clk_after_21_23.to_csv( loc_ftr + "\\clk_after_21_23.csv", index = False )

del clk_18_20['clk_16_20']
del clk_21_23['clk_16_20']

## <span style="color:green"> **Recibio clikcs entre 21 hs y 3 hs (Noche)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
clk_18_20['clk_21_3'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 20)
clk_21_23['clk_21_3'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 20)

clk_night_18_20 = rh_18_20.copy()
clk_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_18_20 = clk_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_night_21_23 = clk_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_night_18_20["clk_21_3"] =  (clk_night_18_20["clk_21_3"] > 1).astype('int8')
clk_night_21_23["clk_21_3"] =  (clk_night_21_23["clk_21_3"] > 1).astype('int8')

clk_night_18_20['me_clk_21_3'] = ( clk_night_18_20.groupby('clk_21_3').transform('count') ) / len(clk_night_18_20['clk_21_3'])
clk_night_21_23['me_clk_21_3'] = ( clk_night_21_23.groupby('clk_21_3').transform('count') ) / len(clk_night_21_23['clk_21_3'])

del clk_night_18_20["clk_21_3"]
del clk_night_21_23["clk_21_3"]

clk_night_18_20.to_csv( loc_ftr + "\\me_clk_night_18_20.csv", index = False )
clk_night_21_23.to_csv( loc_ftr + "\\me_clk_night_21_23.csv", index = False )

del clk_18_20['clk_21_3']
del clk_21_23['clk_21_3']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [36]:
clk_18_20['clk_21_3'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 20)
clk_21_23['clk_21_3'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 4) | ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 20)

clk_night_18_20 = rh_18_20.copy()
clk_night_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_21_3':'sum'}).reset_index()

clk_night_18_20 = clk_night_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_night_21_23 = clk_night_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_night_18_20["clk_21_3"] =  (clk_night_18_20["clk_21_3"] > 1).astype('int8')
clk_night_21_23["clk_21_3"] =  (clk_night_21_23["clk_21_3"] > 1).astype('int8')

clk_night_18_20.to_csv( loc_ftr + "\\clk_night_18_20.csv", index = False )
clk_night_21_23.to_csv( loc_ftr + "\\clk_night_21_23.csv", index = False )

del clk_18_20['clk_21_3']
del clk_21_23['clk_21_3']

## <span style="color:green"> **Recibio clicks entre 11 hs y 15 hs (Medio dia)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
clk_18_20['clk_11_15'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 16) 
clk_21_23['clk_11_15'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 16)

clk_midday_18_20 = rh_18_20.copy()
clk_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_18_20 = clk_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_midday_21_23 = clk_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_midday_18_20["clk_11_15"] =  (clk_midday_18_20["clk_11_15"] > 1).astype('int8')
clk_midday_21_23["clk_11_15"] =  (clk_midday_21_23["clk_11_15"] > 1).astype('int8')

clk_midday_18_20['me_clk_11_15'] = ( clk_midday_18_20.groupby('clk_11_15').transform('count') ) / len(clk_midday_18_20['clk_11_15'])
clk_midday_21_23['me_clk_11_15'] = ( clk_midday_21_23.groupby('clk_11_15').transform('count') ) / len(clk_midday_21_23['clk_11_15'])

del clk_midday_18_20["clk_11_15"]
del clk_midday_21_23["clk_11_15"]

clk_midday_18_20.to_csv( loc_ftr + "\\me_clk_midday_18_20.csv", index = False )
clk_midday_21_23.to_csv( loc_ftr + "\\me_clk_midday_21_23.csv", index = False )

del clk_18_20['clk_11_15']
del clk_21_23['clk_11_15']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [38]:
clk_18_20['clk_11_15'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 16) 
clk_21_23['clk_11_15'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 10) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 16)

clk_midday_18_20 = rh_18_20.copy()
clk_midday_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_11_15':'sum'}).reset_index()

clk_midday_18_20 = clk_midday_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_midday_21_23 = clk_midday_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_midday_18_20["clk_11_15"] =  (clk_midday_18_20["clk_11_15"] > 1).astype('int8')
clk_midday_21_23["clk_11_15"] =  (clk_midday_21_23["clk_11_15"] > 1).astype('int8')

clk_midday_18_20.to_csv( loc_ftr + "\\clk_midday_18_20.csv", index = False )
clk_midday_21_23.to_csv( loc_ftr + "\\clk_midday_21_23.csv", index = False )

del clk_18_20['clk_11_15']
del clk_21_23['clk_11_15']

## <span style="color:green"> **Recibio clicks entre 4 hs y 10 hs (Mañana)** </span> 

## <span style="color:Orange"> **Mean Encoding** </span> 

In [31]:
clk_18_20['clk_4_10'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 11) 
clk_21_23['clk_4_10'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 11)

clk_morn_18_20 = rh_18_20.copy()
clk_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_18_20 = clk_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_morn_21_23 = clk_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_morn_18_20["ins_4_10"] =  (clk_morn_18_20["clk_4_10"] > 1).astype('int8')
clk_morn_21_23["ins_4_10"] =  (clk_morn_21_23["clk_4_10"] > 1).astype('int8')

clk_morn_18_20['me_ins_4_10'] = ( clk_morn_18_20.groupby('ins_4_10').transform('count') ) / len(clk_morn_18_20['ins_4_10'])
clk_morn_21_23['me_ins_4_10'] = ( clk_morn_21_23.groupby('ins_4_10').transform('count') ) / len(clk_morn_21_23['ins_4_10'])

del clk_morn_18_20["ins_4_10"]
del clk_morn_21_23["ins_4_10"]

clk_morn_18_20.to_csv( loc_ftr + "\\me_clk_morn_18_20.csv", index = False )
clk_morn_21_23.to_csv( loc_ftr + "\\me_clk_morn_21_23.csv", index = False )

del clk_18_20['clk_4_10']
del clk_21_23['clk_4_10']

## <span style="color:Orange"> **One-hot Encoding** </span> 

In [39]:
clk_18_20['clk_4_10'] = ( pd.to_datetime( clk_18_20['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_18_20['date'] ).dt.hour < 11) 
clk_21_23['clk_4_10'] = ( pd.to_datetime( clk_21_23['date'] ).dt.hour > 3) & ( pd.to_datetime( clk_21_23['date'] ).dt.hour < 11)

clk_morn_18_20 = rh_18_20.copy()
clk_morn_21_23 = rh_21_23.copy()

hour_mode_18_20 = clk_18_20.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()
hour_mode_21_23 = clk_21_23.groupby('ref_hash').agg({'clk_4_10':'sum'}).reset_index()

clk_morn_18_20 = clk_morn_18_20.merge( hour_mode_18_20, how = "left", on = "ref_hash" )
clk_morn_21_23 = clk_morn_21_23.merge( hour_mode_21_23, how = "left", on = "ref_hash" )

clk_morn_18_20["ins_4_10"] =  (clk_morn_18_20["clk_4_10"] > 1).astype('int8')
clk_morn_21_23["ins_4_10"] =  (clk_morn_21_23["clk_4_10"] > 1).astype('int8')

clk_morn_18_20.to_csv( loc_ftr + "\\clk_morn_18_20.csv", index = False )
clk_morn_21_23.to_csv( loc_ftr + "\\clk_morn_21_23.csv", index = False )

del clk_18_20['clk_4_10']
del clk_21_23['clk_4_10']